My current objective is to get all 30 of the house datasets to have the same set of activities. This can be accomplished by looking at the intersection of all of the individual house activites. I would like to do this so that I can train a model on a subset of all house data, and have the model make predictions for every house. This will likely be a more challenging problem. 

In [1]:
import pickle
import numpy as np

I wrote a script on my gcloud compute machine that read all unique activities and pickled it.

In [2]:
activities = pickle.load(open('activities.pkl', 'rb'))

In [3]:
unique_targets = []
for value in activities.values():
    for activity in value.values:
        if activity not in unique_targets:
            unique_targets.append(activity)

In [4]:
unique_targets = np.array(unique_targets).flatten()

In [5]:
unique_targets.shape

(47,)

In [6]:
unique_targets

array(['Toilet', 'Go_To_Sleep', 'Eat_Lunch', 'Drink', 'Dress',
       'Enter_Home', 'Entertain_Guests', 'Sleep_Out_Of_Bed', 'Phone',
       'Cook_Dinner', 'Cook_Breakfast', 'Other_Activity', 'Eat', 'Sleep',
       'Cook_Lunch', 'Watch_TV', 'Bed_Toilet_Transition', 'Step_Out',
       'Wash_Dishes', 'Bathe', 'Cook', 'Evening_Meds', 'Personal_Hygiene',
       'Morning_Meds', 'Wash_Breakfast_Dishes', 'Read',
       'Wash_Dinner_Dishes', 'Groom', 'Leave_Home', 'Relax',
       'Eat_Breakfast', 'Eat_Dinner', 'Wake_Up', 'Work_At_Table',
       'Wash_Lunch_Dishes', 'Work', 'Take_Medicine', 'Exercise', 'Nap',
       'Work_On_Computer', 'Work_At_Desk', 'Laundry', 'r1.Cook_Breakfast',
       'r1.Sleep', 'r2.Dress', 'r2.Eat_Breakfast', 'r2.Personal_Hygiene'],
      dtype=object)

Now I'll group similar categories together. I manually ensured that each of the similar categories contain a category from each csh house database. I could've wrote a script but that wouldn't save me time. The Only target that isn't in each category is the "Meds" target, and it isn't in csh 124, 127, or 130. Considering my project scope is on providing live streamed assisted living, I'm just going to drop those tables since the unique targets are less relevant to me, thus I will be working with 27 house tables, all in the homes database. 

In [2]:
target_dict = {'Toilet': ['Toilet', 'Bed_Toilet_Transition'],
               'Sleep': ['Go_To_Sleep', 'Sleep', 'Nap', 'r1.Sleep', 'Sleep_Out_Of_Bed'], 
               'Eat': ['Eat_Lunch', 'Eat', 'Eat_Breakfast', 'Eat_Dinner', 'r2.Eat_Breakfast', 'Drink'],
               'Dress': ['Dress', 'r2.Dress'],
               'Enter_Home': ['Enter_Home'],
               'Entertainment': ['Entertain_Guests', 'Phone', 'Watch_TV', 'Read', 'Relax'],
               'Cook':['Cook_Dinner', 'Cook_Breakfast', 'Cook_Lunch', 'Cook', 'r1.Cook_Breakfast'], 
               'Other_Activity': ['Other_Activity', 'Wake_Up', 'Exercise', 'Step_Out', 'Leave_Home', ], 
               'Clean': ['Wash_Dishes', 'Wash_Dinner_Dishes', 'Wash_Breakfast_Dishes', 'Wash_Lunch_Dishes', 'Laundry'],
               'Personal_Hygeine': ['Bathe', 'Groom', 'Personal_Hygiene', 'r2.Personal_Hygiene'], 
               'Meds' : ['Evening_Meds', 'Morning_Meds', 'Take_Medicine'],
               'Work': ['Work_At_Table', 'Work', 'Work_On_Computer', 'Work_At_Desk']}

In [8]:
all_key_length = 0
for value in target_dict.values():
    all_key_length += len(value)

In [9]:
all_key_length

47

In [10]:
targets_not_in = []
# for target in unique_targets:
for key, value in activities.items():
    if ('Work_On_Computer' == value).any().values[0]:
        continue
    else:
        print(key)

101
102
103
105
109
115
123
127
128
129


Great! I have a reduced list of activities, I just have to drop 124, 127, and 130.

In [7]:
import numpy as np
csvs = np.arange(101, 131)
del_csvs = np.array([124, 127, 130])

In [8]:
csvs = np.setdiff1d(csvs, del_csvs)

In [13]:
sql_cases = ''
for key, value in target_dict.items():
    sql_cases += 'WHEN activity in {0} THEN {1} \n'.format(tuple(value), "'{}'".format(key))

In [14]:
print(sql_cases)

WHEN activity in ('Toilet', 'Bed_Toilet_Transition') THEN 'Toilet' 
WHEN activity in ('Go_To_Sleep', 'Sleep', 'Nap', 'r1.Sleep', 'Sleep_Out_Of_Bed') THEN 'Sleep' 
WHEN activity in ('Eat_Lunch', 'Eat', 'Eat_Breakfast', 'Eat_Dinner', 'r2.Eat_Breakfast', 'Drink') THEN 'Eat' 
WHEN activity in ('Dress', 'r2.Dress') THEN 'Dress' 
WHEN activity in ('Enter_Home',) THEN 'Enter_Home' 
WHEN activity in ('Entertain_Guests', 'Phone', 'Watch_TV', 'Read', 'Relax') THEN 'Entertainment' 
WHEN activity in ('Cook_Dinner', 'Cook_Breakfast', 'Cook_Lunch', 'Cook', 'r1.Cook_Breakfast') THEN 'Cook' 
WHEN activity in ('Other_Activity', 'Wake_Up', 'Exercise', 'Step_Out', 'Leave_Home') THEN 'Other_Activity' 
WHEN activity in ('Wash_Dishes', 'Wash_Dinner_Dishes', 'Wash_Breakfast_Dishes', 'Wash_Lunch_Dishes', 'Laundry') THEN 'Clean' 
WHEN activity in ('Bathe', 'Groom', 'Personal_Hygiene', 'r2.Personal_Hygiene') THEN 'Personal_Hygeine' 
WHEN activity in ('Evening_Meds', 'Morning_Meds', 'Take_Medicine') THEN 'Meds' 

In [15]:
# That saved alot of typing!

In [9]:
query = """select *,
    case 
        WHEN activity in ('Toilet', 'Bed_Toilet_Transition') THEN 'Toilet' 
        WHEN activity in ('Go_To_Sleep', 'Sleep', 'Nap', 'r1.Sleep', 'Sleep_Out_Of_Bed') THEN 'Sleep' 
        WHEN activity in ('Eat_Lunch', 'Eat', 'Eat_Breakfast', 'Eat_Dinner', 'r2.Eat_Breakfast', 'Drink') THEN 'Eat' 
        WHEN activity in ('Dress', 'r2.Dress') THEN 'Dress' 
        WHEN activity in ('Enter_Home') THEN 'Enter_Home' 
        WHEN activity in ('Entertain_Guests', 'Phone', 'Watch_TV', 'Read', 'Relax') THEN 'Entertainment' 
        WHEN activity in ('Cook_Dinner', 'Cook_Breakfast', 'Cook_Lunch', 'Cook', 'r1.Cook_Breakfast') THEN 'Cook' 
        WHEN activity in ('Other_Activity', 'Wake_Up', 'Exercise', 'Step_Out', 'Leave_Home') THEN 'Other_Activity' 
        WHEN activity in ('Wash_Dishes', 'Wash_Dinner_Dishes', 'Wash_Breakfast_Dishes', 'Wash_Lunch_Dishes', 'Laundry') THEN 'Clean' 
        WHEN activity in ('Bathe', 'Groom', 'Personal_Hygiene', 'r2.Personal_Hygiene') THEN 'Personal_Hygeine' 
        WHEN activity in ('Evening_Meds', 'Morning_Meds', 'Take_Medicine') THEN 'Meds' 
        WHEN activity in ('Work_At_Table', 'Work', 'Work_On_Computer', 'Work_At_Desk') THEN 'Work' 
        else 'not_in_activities'
    end as reduced_activity
from {};"""

In [10]:
from personal_connection import create_connection
import pandas as pd


In [11]:
conn = create_connection()

In [12]:
tables = ['csh{}'.format(i) for i in csvs]

In [20]:
result = pd.read_sql_query(query.format('csh110'), conn, chunksize=1000)

In [71]:
big_df = pd.DataFrame()
for df in result:
    print(df)

     lastSensorEventHours  lastSensorEventSeconds  lastSensorDayOfWeek  \
0                    13.0                 49922.0                  5.0   
1                    13.0                 49923.0                  5.0   
2                    13.0                 49924.0                  5.0   
3                    13.0                 49925.0                  5.0   
4                    13.0                 49925.0                  5.0   
..                    ...                     ...                  ...   
995                  17.0                 62453.0                  5.0   
996                  17.0                 62455.0                  5.0   
997                  17.0                 62457.0                  5.0   
998                  17.0                 62460.0                  5.0   
999                  17.0                 62462.0                  5.0   

     windowDuration  timeSinceLastSensorEvent  prevDominantSensor1  \
0              22.0                      

     lastSensorEventHours  lastSensorEventSeconds  lastSensorDayOfWeek  \
0                     6.0                 23431.0                  6.0   
1                     6.0                 23435.0                  6.0   
2                     6.0                 23437.0                  6.0   
3                     6.0                 23440.0                  6.0   
4                     6.0                 23440.0                  6.0   
..                    ...                     ...                  ...   
995                   9.0                 33903.0                  6.0   
996                   9.0                 33904.0                  6.0   
997                   9.0                 33918.0                  6.0   
998                   9.0                 33920.0                  6.0   
999                   9.0                 33976.0                  6.0   

     windowDuration  timeSinceLastSensorEvent  prevDominantSensor1  \
0           11293.0                     1

     lastSensorEventHours  lastSensorEventSeconds  lastSensorDayOfWeek  \
0                    13.0                 48761.0                  6.0   
1                    13.0                 48800.0                  6.0   
2                    13.0                 48801.0                  6.0   
3                    13.0                 48804.0                  6.0   
4                    13.0                 48805.0                  6.0   
..                    ...                     ...                  ...   
995                  21.0                 75812.0                  6.0   
996                  21.0                 75812.0                  6.0   
997                  21.0                 75813.0                  6.0   
998                  21.0                 75814.0                  6.0   
999                  21.0                 75814.0                  6.0   

     windowDuration  timeSinceLastSensorEvent  prevDominantSensor1  \
0            1902.0                      

     lastSensorEventHours  lastSensorEventSeconds  lastSensorDayOfWeek  \
0                     9.0                 35306.0                  0.0   
1                     9.0                 35306.0                  0.0   
2                     9.0                 35307.0                  0.0   
3                     9.0                 35308.0                  0.0   
4                     9.0                 35319.0                  0.0   
..                    ...                     ...                  ...   
995                  11.0                 40788.0                  0.0   
996                  11.0                 40792.0                  0.0   
997                  11.0                 40792.0                  0.0   
998                  11.0                 40792.0                  0.0   
999                  11.0                 40793.0                  0.0   

     windowDuration  timeSinceLastSensorEvent  prevDominantSensor1  \
0              23.0                      

     lastSensorEventHours  lastSensorEventSeconds  lastSensorDayOfWeek  \
0                    18.0                 68326.0                  0.0   
1                    18.0                 68327.0                  0.0   
2                    18.0                 68332.0                  0.0   
3                    18.0                 68333.0                  0.0   
4                    18.0                 68334.0                  0.0   
..                    ...                     ...                  ...   
995                   0.0                    10.0                  1.0   
996                   0.0                    12.0                  1.0   
997                   0.0                    18.0                  1.0   
998                   0.0                    19.0                  1.0   
999                   0.0                    22.0                  1.0   

     windowDuration  timeSinceLastSensorEvent  prevDominantSensor1  \
0              55.0                      

...

     lastSensorEventHours  lastSensorEventSeconds  lastSensorDayOfWeek  \
0                    23.0                 83283.0                  6.0   
1                    23.0                 83284.0                  6.0   
2                    23.0                 83303.0                  6.0   
3                    23.0                 83304.0                  6.0   
4                    23.0                 83311.0                  6.0   
..                    ...                     ...                  ...   
995                   7.0                 26402.0                  0.0   
996                   7.0                 26407.0                  0.0   
997                   7.0                 26428.0                  0.0   
998                   7.0                 26440.0                  0.0   
999                   7.0                 26499.0                  0.0   

     windowDuration  timeSinceLastSensorEvent  prevDominantSensor1  \
0             361.0                      

     lastSensorEventHours  lastSensorEventSeconds  lastSensorDayOfWeek  \
0                     1.0                  5951.0                  1.0   
1                     1.0                  5951.0                  1.0   
2                     1.0                  5954.0                  1.0   
3                     1.0                  5954.0                  1.0   
4                     1.0                  5954.0                  1.0   
..                    ...                     ...                  ...   
995                   9.0                 35622.0                  1.0   
996                   9.0                 35624.0                  1.0   
997                   9.0                 35625.0                  1.0   
998                   9.0                 35632.0                  1.0   
999                   9.0                 35633.0                  1.0   

     windowDuration  timeSinceLastSensorEvent  prevDominantSensor1  \
0             201.0                      

     lastSensorEventHours  lastSensorEventSeconds  lastSensorDayOfWeek  \
0                    23.0                 84200.0                  1.0   
1                    23.0                 84202.0                  1.0   
2                    23.0                 84202.0                  1.0   
3                    23.0                 84220.0                  1.0   
4                    23.0                 84220.0                  1.0   
..                    ...                     ...                  ...   
995                   5.0                 19051.0                  2.0   
996                   5.0                 20959.0                  2.0   
997                   5.0                 20960.0                  2.0   
998                   5.0                 20962.0                  2.0   
999                   5.0                 20964.0                  2.0   

     windowDuration  timeSinceLastSensorEvent  prevDominantSensor1  \
0             393.0                      

     lastSensorEventHours  lastSensorEventSeconds  lastSensorDayOfWeek  \
0                    14.0                 52666.0                  2.0   
1                    14.0                 52909.0                  2.0   
2                    14.0                 52922.0                  2.0   
3                    14.0                 52937.0                  2.0   
4                    14.0                 52937.0                  2.0   
..                    ...                     ...                  ...   
995                  19.0                 71318.0                  2.0   
996                  19.0                 71318.0                  2.0   
997                  19.0                 71319.0                  2.0   
998                  19.0                 71319.0                  2.0   
999                  19.0                 71341.0                  2.0   

     windowDuration  timeSinceLastSensorEvent  prevDominantSensor1  \
0             916.0                      

KeyboardInterrupt: 

In [13]:
import os

In [20]:
for chunk in pd.read_sql_query(query.format('csh110') , conn, chunksize=10000):
    chunk.to_csv(os.path.join('csh110' + ".csv"), mode='a',sep=',',encoding='utf-8')

In [21]:
csh110 = pd.read_csv('csh110.csv')

/home/dyllanjr/anaconda3/envs/metis/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [25]:
csh110 = csh110[csh110.lastSensorEventHours != 'lastSensorEventHours']
csh110[csh110['sensorElTime-WorkArea'] == 'Other_Activity']


,Unnamed: 0,lastSensorEventHours,lastSensorEventSeconds,lastSensorDayOfWeek,windowDuration,timeSinceLastSensorEvent,prevDominantSensor1,prevDominantSensor2,lastSensorID,lastSensorLocation,...,sensorElTime-DiningRoom,sensorElTime-Hall,sensorElTime-Ignore,sensorElTime-Kitchen,sensorElTime-LivingRoom,sensorElTime-Office,sensorElTime-OutsideDoor,sensorElTime-WorkArea,activity,reduced_activity


In [23]:
#okay I'm ready to run this query on each table! The following loop allows
#me to write to a csv without running out of memory.

In [14]:
%pwd

'/home/dyllanjr/Documents/Classification-M2/notebooks/model_building'

In [15]:
%cd /home/dyllanjr/Documents/Classification-M2/data/processed

/home/dyllanjr/Documents/Classification-M2/data/processed


In [16]:
for table in tables:
    for chunk in pd.read_sql_query(query.format(table) , conn, chunksize=10000):
        chunk.to_csv(os.path.join(table + ".csv"), mode='a',sep=',',encoding='utf-8')

KeyboardInterrupt: 

In [17]:
#I interupted the load after 10 house datasets, I didn't want to deal with so much
#data on my local pc.
#now i just need to do some cleaning

In [21]:
for table in tables:
    df = pd.read_csv(table+'.csv')
    df = df[df.lastSensorEventHours != 'lastSensorEventHours']
    df.to_csv(table+'.csv')
    del df

/home/dyllanjr/anaconda3/envs/metis/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


FileNotFoundError: [Errno 2] File csh111.csv does not exist: 'csh111.csv'

In [22]:
df = pd.read_csv('csh108.csv')
df[df['sensorElTime-WorkArea'] == 'Other_Activity']

/home/dyllanjr/anaconda3/envs/metis/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,Unnamed: 0,Unnamed: 0.1,lastSensorEventHours,lastSensorEventSeconds,lastSensorDayOfWeek,windowDuration,timeSinceLastSensorEvent,prevDominantSensor1,prevDominantSensor2,lastSensorID,...,sensorElTime-DiningRoom,sensorElTime-Hall,sensorElTime-Ignore,sensorElTime-Kitchen,sensorElTime-LivingRoom,sensorElTime-Office,sensorElTime-OutsideDoor,sensorElTime-WorkArea,activity,reduced_activity


In [8]:
#my kernel died, going to try with the remaining csvs
csvs = [129, 128,126,125,123,122,121,120,119,118,117,116,115,114,113]
tables = ['csh{}'.format(i) for i in csvs]

In [ ]:
for table in tables:
    df = pd.read_csv(table+'.csv')
    df = df[df.lastSensorEventHours != 'lastSensorEventHours']
    df.to_csv(table+'.csv')
    del df

/home/dyllanjr/anaconda3/envs/metis/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,40) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/home/dyllanjr/anaconda3/envs/metis/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,40) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [12]:
csvs = [120, 122]
tables = ['csh{}'.format(i) for i in csvs]
for table in tables:
    df = pd.read_csv(table + '.csv')
    df = df[df.lastSensorEventHours != 'lastSensorEventHours']
    df.to_csv(table+'.csv')
    del df

In [ ]:
df = pd.read_csv('csh113.csv')
df = df[df.lastSensorEventHours != 'lastSensorEventHours']
df.to_csv('csh113.csv')
del df

/home/dyllanjr/anaconda3/envs/metis/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,40) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Variable            Type        Data/Info
-----------------------------------------
create_connection   function    <function create_connection at 0x7f2c605fa550>
pd                  module      <module 'pandas' from '/h<...>ages/pandas/__init__.py'>
